<a href="https://colab.research.google.com/github/JZhouSaclay/DataChallenge/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import holidays
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from torch import nn as nn
import torch.optim as optim
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import joblib

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def fill_with_cyclic_mean(df, col):
    for index in df[df[col].isnull()].index:  # parcourir les données manquantes
        # obtenir les informations temporaires de la donnée manquante
        month = df.loc[index, 'Month']
        day = df.loc[index, 'Day']
        hour = df.loc[index, 'Hour']

        # trouver les valeurs de même instant des autres années
        similar_time_data = df[(df['Month'] == month) &
                               (df['Day'] == day) &
                               (df['Hour'] == hour) &
                               (~df[col].isnull())][col]

        # calculer une moyenne
        if not similar_time_data.empty:
            df.at[index, col] = similar_time_data.mean()

def dataProcessing(data_name):
    public_df = pd.read_csv(f'./{data_name}.csv')

    public_df['date'] = pd.to_datetime(public_df['date'], utc=True)
    public_df['date_utc'] = public_df['date'].dt.tz_convert('UTC')

    # les infos de temps basique
    public_df['Month'] = public_df['date_utc'].dt.month
    public_df['Year'] = public_df['date_utc'].dt.year
    public_df['Day'] = public_df['date_utc'].dt.day
    public_df['Hour'] = public_df['date_utc'].dt.hour
    public_df['Week'] = public_df['date'].dt.isocalendar().week
    public_df['Weekday'] = public_df['date_utc'].dt.weekday
    public_df['toy'] = public_df['date'].dt.dayofyear
    public_df['Minute'] = public_df['date_utc'].dt.minute
    public_df['Half_Hour'] = public_df['Hour'] * 2 + (public_df['Minute'] // 30)

    # les vacances et jours fériés
    fr_holidays = holidays.France()
    public_df['is_holiday'] = public_df['date_utc'].dt.date.apply(lambda x: x in fr_holidays)

    public_df['is_summer_vacation'] = public_df['date_utc'].apply(
        lambda x: (x.month == 8) or (x.month == 6 and x.day >= 10) or (x.month == 7)
    )
    public_df['vacances'] = public_df['date_utc'].apply(
        lambda x: (x.month == 8 and x.day >= 1 and x.day <= 15) or
                  (x.month == 12 and x.day >= 25) or
                  (x.month == 1 and x.day <= 2)
    )
    public_df['big_holiday'] = public_df['date_utc'].apply(
        lambda x: (x.month == 5 and x.day == 1) or
                  (x.month == 7 and x.day == 14) or
                  (x.month == 12 and x.day == 25) or
                  (x.month == 1 and x.day == 1)
    )

    # pandémie
    pandemic_periods = [
        ('2020-03-17', '2020-05-10'),
        ('2020-10-30', '2020-12-14'),
        ('2021-02-27', '2021-02-28'),
        ('2021-03-06', '2021-03-07'),
        ('2021-04-01', '2021-04-04')
    ]
    public_df['is_pandemic'] = False
    for start, end in pandemic_periods:
        public_df['is_pandemic'] |= public_df['date_utc'].between(start, end)

    public_df['Season'] = public_df['Month'].apply(
        lambda month: 0 if month in [3,4, 5] else
                      1 if month in [6, 7, 8,9] else
                      2 if month in [10, 11] else
                      3
    )

    # type de jours dans la semaine
    public_df['is_weekend'] = public_df['Weekday'].apply(lambda x: x in [5, 6])
    public_df['is_workday'] = public_df['Weekday'].apply(lambda x: x in [1, 2, 3])
    public_df['is_transday'] = public_df['Weekday'].apply(lambda x: x in [0, 4])
    public_df['is_vacation'] = public_df['Week'].apply(lambda x: x in range(30, 35))

    # sin/cos encoding de 48 dimensions
    time_variables = {
        'Half_Hour': 48,      #48 intervalles de demi-heure dans une journée
        'Weekday': 7,         # 7 j/semaine
        'Month': 12,
        'Season': 4,
        'toy': 365,
    }
    max_k = 48 # dim max

    # on utilise dict pour tout enregistrer, puis concaténer, pour éviter de trop modifier le dataframe
    fourier_features = {}

    for var, period in time_variables.items():
        for k in range(1, max_k + 1):
            fourier_features[f'sin_{var}_k{k}'] = np.sin(2 * np.pi * k * public_df[var] / period)
            fourier_features[f'cos_{var}_k{k}'] = np.cos(2 * np.pi * k * public_df[var] / period)

    public_df = pd.concat([public_df, pd.DataFrame(fourier_features)], axis=1)

    return public_df


def weatherProcessing(data_name):
    weather = pd.read_parquet(f'./{data_name}.parquet')
    weather['date'] = pd.to_datetime(weather['date'], utc=True)
    variables = ['t', 'u', 'td', 'ff', 'dd']

    # on supprime celui qui a trop de données manquantes, impossible de remplir
    weather_metropole = weather[weather['nom_epci'] != "Métropole Toulon-Provence-Méditerranée"]

    weather_avg_metropole = weather_metropole.groupby(['nom_epci', 'date'])[variables].mean().reset_index()
    weather_pivot_metropole = weather_avg_metropole.pivot_table(index='date', columns='nom_epci', values=variables)
    weather_pivot_metropole = weather_pivot_metropole.interpolate(method='linear', limit_direction='both')
    weather_pivot_metropole.index = pd.to_datetime(weather_pivot_metropole.index, utc=True)
    weather_resampled_metropole = weather_pivot_metropole.resample('30min').interpolate(method='linear')

    # niveau Région
    weather_avg = weather.groupby(['nom_reg', 'date'])[variables].mean().reset_index()
    weather_pivot = weather_avg.pivot_table(index='date', columns='nom_reg', values=variables)
    weather_pivot = weather_pivot.interpolate(method='linear', limit_direction='both')
    weather_pivot.index = pd.to_datetime(weather_pivot.index, utc=True)
    weather_resampled = weather_pivot.resample('30min').interpolate(method='linear')

    # ajout de nouvelles colonnes correspondantes à chaque variables de météo
    weather_resampled_metropole.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in weather_resampled_metropole.columns]
    weather_resampled.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in weather_resampled.columns]

    # reset index avec date
    if 'date' not in weather_resampled_metropole.columns:
        weather_resampled_metropole.reset_index(inplace=True)
    if 'date' not in weather_resampled.columns:
        weather_resampled.reset_index(inplace=True)

    # on fait d'abord pour ville, puis région, ensuite concat
    weather_df = pd.merge(weather_resampled_metropole, weather_resampled, on='date', how='outer')

    return weather_df

class MLPWithAttentionAll(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, output_dim=25, num_layers=3, dropout_rate=0.2):
        super(MLPWithAttentionAll, self).__init__()

        self.layers = nn.ModuleList()
        self.batchnorms = nn.ModuleList()
        self.attentions = nn.ModuleList()

        # input
        self.layers.append(nn.Linear(input_dim, hidden_dim))
        self.batchnorms.append(nn.BatchNorm1d(hidden_dim))
        self.attentions.append(nn.Linear(hidden_dim, 1))  # self.attention

        # hidden
        for _ in range(num_layers - 1):  # num_layers
            self.layers.append(nn.Linear(hidden_dim, hidden_dim))
            self.batchnorms.append(nn.BatchNorm1d(hidden_dim))
            self.attentions.append(nn.Linear(hidden_dim, 1))

        # output
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        for i in range(len(self.layers)):
            x = F.relu(self.layers[i](x))
            x = self.batchnorms[i](x)  # batch normalisation
            attn_weights = torch.sigmoid(self.attentions[i](x))
            x = x * attn_weights
            x = self.dropout(x)

        x = self.output_layer(x)  # les 25 colonnes en une fois
        return x

def RMSE(y_pred, y_true):
    return torch.sqrt(((y_pred - y_true)**2).mean())



In [3]:
weather_data = weatherProcessing("meteo")

In [5]:
target_columns = ['France','Auvergne-Rhône-Alpes','Bourgogne-Franche-Comté','Bretagne','Centre-Val de Loire',
 'Grand Est','Hauts-de-France','Normandie','Nouvelle-Aquitaine','Occitanie','Pays de la Loire',"Provence-Alpes-Côte d'Azur",
 'Île-de-France','Montpellier Méditerranée Métropole','Métropole Européenne de Lille','Métropole Grenoble-Alpes-Métropole',
 "Métropole Nice Côte d'Azur",'Métropole Rennes Métropole','Métropole Rouen Normandie',"Métropole d'Aix-Marseille-Provence",
 'Métropole de Lyon','Métropole du Grand Nancy','Métropole du Grand Paris','Nantes Métropole','Toulouse Métropole']

In [6]:
import pandas as pd
import torch
import joblib
import numpy as np
#from train import dataProcessing, MLPWithAttentionAll

# 设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 读取测试数据
test_data = dataProcessing("test")
test_data = test_data.merge(weather_data, on='date', how='left')
test_data = test_data.select_dtypes(exclude=['datetime64[ns, UTC]'])

drop_features = [ 'Day', 'Hour', 'Week', 'toy', 'Minute', 'Half_Hour']
test_data = test_data.drop(columns=drop_features, errors='ignore')

# 载入标准化器
scaler_X = joblib.load("scaler_X.pkl")
scaler_y = joblib.load("scaler_y.pkl")
scale_tensor = torch.tensor(scaler_y.scale_, dtype=torch.float32, device=device)
mean_tensor = torch.tensor(scaler_y.mean_, dtype=torch.float32, device=device)


# 加载模型
# 确保 `input_dim` 与训练时一致
input_dim = joblib.load("scaler_X.pkl").n_features_in_
model = MLPWithAttentionAll(input_dim).to(device)

model.load_state_dict(torch.load("model.pth"))
model.eval()

# 预处理测试数据
X_test = test_data.values
X_test_scaled = scaler_X.transform(X_test)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32, device=device)

# 预测
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor)
    y_pred_real = y_pred_scaled * scale_tensor + mean_tensor

original_pred_df = pd.read_csv("pred.csv")
pred_df = pd.DataFrame(y_pred_real.cpu().numpy(), columns=["pred_" + c for c in target_columns])

for col in pred_df.columns:
    if col in original_pred_df.columns:
        original_pred_df[col] = pred_df[col]
    else:
        print(f" {col} skipped")

original_pred_df.to_csv("pred_final.csv", index=False)
print("Done, pred.csv！")


<ipython-input-6-a1df7760d805>:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


Done, pred.csv！
